In [9]:
%pip install yfinance pandas numpy matplotlib seaborn nltk textblob tensorflow scikit-learn newsapi-python yahoo_fin GoogleNews



   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- -------------------------- 1/3 [dateparser]
   ------------- ------

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from textblob import TextBlob
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


In [3]:
data = yf.download('AAPL', start='2015-01-01', end='2024-01-01')
data = data[['Close']]
data.dropna(inplace=True)


YF.download() has changed argument auto_adjust default to True


Failed to get ticker 'AAPL' reason: Failed to perform, curl: (28) Resolving timed out after 10006 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.
[*********************100%***********************]  1 of 1 completed


In [10]:
from GoogleNews import GoogleNews
from textblob import TextBlob
import pandas as pd
import datetime

# Initialize GoogleNews
googlenews = GoogleNews(lang='en', period='7d')  # last 7 days
googlenews.search('Apple stock')

# Get results
results = googlenews.results()

sentiments = []
dates = []

for article in results:
    text = article['title'] + '. ' + article.get('desc', '')
    blob = TextBlob(text)
    sentiments.append(blob.sentiment.polarity)

    date_str = article.get('date', 'now')
    try:
        date_obj = datetime.datetime.strptime(date_str, '%b %d, %Y')
    except:
        date_obj = datetime.datetime.today()
    dates.append(date_obj.date())

# Create DataFrame
sentiment_df = pd.DataFrame({
    'date': dates,
    'sentiment': sentiments
})

# Group by date and calculate average sentiment
sentiment_df = sentiment_df.groupby('date').mean().reset_index()
print(sentiment_df)


<urlopen error [Errno 11001] getaddrinfo failed>
Empty DataFrame
Columns: [date, sentiment]
Index: []
